<a href="https://colab.research.google.com/github/KNagaveni/3D-MODELS/blob/main/Multi_Agent_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# === Required Libraries ===
import os
import json
import datetime
from typing import Dict, Any
import streamlit as st
import fitz  # PyMuPDF
import google.generativeai as genai

In [24]:
# === Google Gemini API Key ===
from getpass import getpass
gemini_key = getpass("Enter your Google Gemini API Key: ")

Enter your Google Gemini API Key: ··········


In [25]:
import google.generativeai as genai
genai.configure(api_key=gemini_key)

In [26]:
# === Helper Function to Parse LLM Response ===
def parse_llm_response(response_text):
    try:
        # Remove markdown formatting if present
        if response_text.startswith("json"):
            response_text = response_text.replace("json", "").replace("   ", "").strip()
        return json.loads(response_text)
    except Exception as e:
        return {"error": str(e), "raw_response": response_text}

# === LLM Call ===
def call_llm(prompt: str) -> Dict[str, Any]:
    print("Sending to LLM:\n", prompt)
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content([prompt])
    return parse_llm_response(response.text)

# === Shared Memory ===
class SharedMemory:
    def __init__(self):
        self.memory = []

    def store(self, data: Dict[str, Any]):
        self.memory.append(data)
        print("Memory updated:", data)

# === Agent: Classifier ===
class ClassifierAgent:
    def classify(self, input_data: str) -> Dict[str, str]:
        prompt = f"""
        You are a classification agent.
        Your job is to analyze the given input and determine:
        1. The format of the input (PDF, Email, or JSON).
        2. The intent of the message (e.g., Invoice, RFQ, Complaint, Regulation, etc.).

        Input:
        {input_data}

        Respond in JSON format:
        {{
          "format": "PDF / Email / JSON",
          "intent": "e.g., Invoice / RFQ / Complaint / Regulation"
        }}
        """
        return call_llm(prompt)

# === Agent: Email ===
class EmailAgent:
    def process(self, email_text: str) -> Dict[str, Any]:
        prompt = f"""
        You are an Email extraction agent.
        Analyze the email content below to extract:
        - Sender name or organization
        - Intent (Invoice, RFQ, Complaint, etc.)
        - Urgency (Low, Medium, High)
        - Summary of the message in 2-3 lines

        Email Content:
        {email_text}

        Respond in JSON format:
        {{
          "sender": "...",
          "intent": "...",
          "urgency": "...",
          "summary": "..."
        }}
        """
        return call_llm(prompt)

# === Agent: JSON ===
class JSONAgent:
    def process(self, json_data: str) -> Dict[str, Any]:
        prompt = f"""
        You are a JSON extraction agent.
        Your job is to:
        - Extract fields from the given JSON.
        - Convert the fields to a predefined schema: {{customer_name, product, quantity, date}}.
        - Flag if any field is missing or suspicious.

        Input JSON:
        {json_data}

        Respond in JSON format:
        {{
          "standardized_data": {{ ... }},
          "missing_fields": [...],
          "anomalies": [...]
        }}
        """
        return call_llm(prompt)

# === Agent: PDF ===
class PDFAgent:
    def process(self, pdf_path: str) -> Dict[str, Any]:
        doc = fitz.open(pdf_path)
        text = "\n".join([page.get_text() for page in doc])
        prompt = f"""
        You are a PDF extraction agent.
        Analyze the text below and extract:
        - Organization or sender
        - Intent (Invoice, Complaint, RFQ, etc.)
        - Summary in 2 lines

        PDF Text:
        {text[:2000]}

        Respond in JSON format:
        {{
          "sender": "...",
          "intent": "...",
          "summary": "..."
        }}
        """
        return call_llm(prompt)

# === Driver Code ===
classifier = ClassifierAgent()
json_agent = JSONAgent()
email_agent = EmailAgent()
pdf_agent = PDFAgent()
memory = SharedMemory()

# === Input Example ===
input_type = input("Enter type (Email / JSON / PDF): ").strip()

if input_type.lower() == "email":
    user_input = input("Paste email content:\n")
    result = classifier.classify(user_input)
    format_ = result.get("format", "Unknown")
    intent = result.get("intent", "Unknown")
    processed = email_agent.process(user_input)
    memory.store({
        "source": format_,
        "type": intent,
        "timestamp": datetime.datetime.now().isoformat(),
        "extracted_data": processed,
        "thread_id": f"thread-{datetime.datetime.now().timestamp()}"
    })
    print("\nProcessed Email Result:", json.dumps(processed, indent=2))

elif input_type.lower() == "json":
    user_input = input("Paste JSON content:\n")
    result = classifier.classify(user_input)
    format_ = result.get("format", "Unknown")
    intent = result.get("intent", "Unknown")
    processed = json_agent.process(user_input)
    memory.store({
        "source": format_,
        "type": intent,
        "timestamp": datetime.datetime.now().isoformat(),
        "extracted_data": processed,
        "thread_id": f"thread-{datetime.datetime.now().timestamp()}"
    })
    print("\nProcessed JSON Result:", json.dumps(processed, indent=2))

elif input_type.lower() == "pdf":
    print("Please upload a PDF file manually")
    uploaded = files.upload()
    for filename in uploaded:
        with fitz.open(filename) as doc:
            pdf_text = "\n".join([page.get_text() for page in doc])
        result = classifier.classify(pdf_text)
        format_ = result.get("format", "Unknown")
        intent = result.get("intent", "Unknown")
        processed = pdf_agent.process(filename)
        memory.store({
            "source": format_,
            "type": intent,
            "timestamp": datetime.datetime.now().isoformat(),
            "extracted_data": processed,
            "thread_id": f"thread-{datetime.datetime.now().timestamp()}"
        })
        print("\nProcessed PDF Result:", json.dumps(processed, indent=2))

else:
    print("Invalid input type. Please choose Email, JSON, or PDF.")


Enter type (Email / JSON / PDF): Email
Paste email content:
 From: john.doe@company.com     Subject: Request for Quote      Hello,     We are looking to purchase 50 laptops for our team. Please send a quote.      Regards,     John Doe
Sending to LLM:
 
        You are a classification agent.
        Your job is to analyze the given input and determine:
        1. The format of the input (PDF, Email, or JSON).
        2. The intent of the message (e.g., Invoice, RFQ, Complaint, Regulation, etc.).

        Input:
         From: john.doe@company.com     Subject: Request for Quote      Hello,     We are looking to purchase 50 laptops for our team. Please send a quote.      Regards,     John Doe

        Respond in JSON format:
        {
          "format": "PDF / Email / JSON",
          "intent": "e.g., Invoice / RFQ / Complaint / Regulation"
        }
        


2025-05-31 15:27:42.124 200 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2509.30ms


Sending to LLM:
 
        You are an Email extraction agent.
        Analyze the email content below to extract:
        - Sender name or organization
        - Intent (Invoice, RFQ, Complaint, etc.)
        - Urgency (Low, Medium, High)
        - Summary of the message in 2-3 lines

        Email Content:
         From: john.doe@company.com     Subject: Request for Quote      Hello,     We are looking to purchase 50 laptops for our team. Please send a quote.      Regards,     John Doe

        Respond in JSON format:
        {
          "sender": "...",
          "intent": "...",
          "urgency": "...",
          "summary": "..."
        }
        


2025-05-31 15:27:43.734 200 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1600.82ms


Memory updated: {'source': 'Unknown', 'type': 'Unknown', 'timestamp': '2025-05-31T15:27:43.736740', 'extracted_data': {'error': 'Expecting value: line 1 column 1 (char 0)', 'raw_response': '```json\n{\n  "sender": "john.doe@company.com",\n  "intent": "RFQ",\n  "urgency": "Low",\n  "summary": "The sender, John Doe, is requesting a quote for the purchase of 50 laptops.  No specific timeframe is mentioned."\n}\n```\n'}, 'thread_id': 'thread-1748705263.736749'}

Processed Email Result: {
  "error": "Expecting value: line 1 column 1 (char 0)",
  "raw_response": "```json\n{\n  \"sender\": \"john.doe@company.com\",\n  \"intent\": \"RFQ\",\n  \"urgency\": \"Low\",\n  \"summary\": \"The sender, John Doe, is requesting a quote for the purchase of 50 laptops.  No specific timeframe is mentioned.\"\n}\n```\n"
}
